In [4]:
import os
print(os.getcwd())
from Garmin_Handler import Overview_File_Reader, Activity_Handler, create_files_list

C:\Users\nicoj\python_projects\Garmin_Handler\Garmin_Handler


ModuleNotFoundError: No module named 'Garmin_Handler'

In [ ]:
# create csv file for each activity
# name will be: "Name of activity" + "Time"
warnings.warn("Bei der Umwandlung gibt es noch Probleme mit der Uhrzeit. Verdacht auf Probleme bei Zeitumstellung-")

from fitparse import FitFile
import pandas as pd

#not working examples
nwa = []

for file in file_list:
    #create empty df
    df = pd.DataFrame(columns=['enhanced_altitude','enhanced_speed','heart_rate','timestamp','temperature'])

    #create lists for target values
    alt_list = []
    speed_list = []
    hr_list = []
    time_list = []
    temp_list = []
    lat_list = []
    lon_list = []
    dist_list = []
    print(file)

    # parse trough file
    for record in FitFile(file).get_messages('record'):
        #print(record)


        # Go through all the data entries in this record
        for record_data in record:


            if record_data.name == "enhanced_altitude":
                alt_list.append(record_data.value)

            elif record_data.name == "enhanced_speed":
                speed_list.append(record_data.value)
            elif record_data.name == "heart_rate":
                hr_list.append(record_data.value)
            elif record_data.name == "timestamp":
                time_list.append(record_data.value)
            elif record_data.name == "temperature":
                temp_list.append(record_data.value)
            elif record_data.name == "position_lat":
                lat_list.append(record_data.value)
            elif record_data.name == "position_long":
                lon_list.append(record_data.value)
            elif record_data.name == "distance":
                dist_list.append(record_data.value)

    # create a dataframe to store all activity data
    df = pd.DataFrame([time_list,dist_list,   speed_list, alt_list,  hr_list , temp_list,lat_list,lon_list]).T
    df.columns=['timestamp',"distance", 'enhanced_speed','enhanced_altitude','heart_rate','temperature', "latitude", "longitude"]

    df["timestamp"] = pd.to_datetime(df["timestamp"]) + pd.to_timedelta(2, unit='h')

    #change of["Datum"] to datetime for making it searchable
    of["Datum"] = pd.to_datetime(of["Datum"])

    #if there is a mutual time in of["Datum"] and the first timestep of the current activity
    # choose activity name like this.
    print(df.head())
    if len(of[of["Datum"]==df["timestamp"][0]]) == 1:
        activity_name = of[of["Datum"]==df["timestamp"][0]]["Aktivitätstyp"].reset_index(drop = True)[0]
        activity_time = of[of["Datum"]==df["timestamp"][0]]["Datum"].reset_index(drop = True)[0]

    # if there is no mutual time in of["Datum"] and the first timestep of the current activity
    # make an entrance in nwa list.
    # and see whether there is exactly one activity listed in of that could match, but with a time error of 20 minutes
    # if so, choose this activity_name 

    # TODO implement a method that could cope with the problem of there being two possible activites in the timeslot.
    elif len(of[of["Datum"]==df["timestamp"][0]]) == 0 :
        nwa.append(file)
        print(nwa)

        if len(of[(of["Datum"]>(df["timestamp"][0]- pd.to_timedelta(20, unit='min')))& (of["Datum"]<(df["timestamp"][0]+ pd.to_timedelta(20, unit='min')))]) == 1:
            del nwa[-1]
            activity_name = of[(of["Datum"]>(df["timestamp"][0]- pd.to_timedelta(20, unit='min')))& (of["Datum"]<(df["timestamp"][0]+ pd.to_timedelta(20, unit='min')))]["Aktivitätstyp"].reset_index(drop = True)[0]
            activity_time = of[(of["Datum"]>(df["timestamp"][0]- pd.to_timedelta(20, unit='min')))& (of["Datum"]<(df["timestamp"][0]+ pd.to_timedelta(20, unit='min')))]["Datum"].reset_index(drop = True)[0]
            print(activity_name)

        if len(of[(of["Datum"]>(df["timestamp"][0]- pd.to_timedelta(20, unit='min')))& (of["Datum"]<(df["timestamp"][0]+ pd.to_timedelta(20, unit='min')))]) == 1:
            del nwa[-1]






    output = pd.to_datetime(activity_time).strftime("%Y%m%d-%H%M%S")
    df.to_csv(os.path.join(target_path, str(activity_name + "_" + output  + ".csv")))




if len(nwa)==0:
    print("keine unhandlebaren Aktivitäten")
if len(nwa)>0:
    print(str(len(nwa)) + " unhandlebare Aktivitäten")
return nwa
